# 02 · Feature Engineering
Clean numeric fields, parse dates, and build features (e.g., ratios & simple aggregates).

In [1]:

import pandas as pd
import numpy as np
from pathlib import Path

CLEAN_CSV = "../data/processed/data_clean.csv"
FEAT_CSV = "../data/processed/features.csv"

df = pd.read_csv(CLEAN_CSV)
df.head()


,symbol,name,exchange,ipoDate,delistingDate,status,URL,salesQ5,salesQ4,salesQ3,...,opCashflowQ5,opCashflowQ4,opCashflowQ3,opCashflowQ2,opCashflowQ1,netCashflowQ5,netCashflowQ4,netCashflowQ3,netCashflowQ2,netCashflowQ1
0,AACQU,Origin Materials Inc - Units (1 Ord Share Clas...,NASDAQ,2020-07-14,2021-06-24,Delisted,https://www.barchart.com/stocks/quotes/ACNDWS/...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AAIN,Arlington Asset Investment Corp,NYSE,2021-07-19,2024-01-30,Delisted,https://www.barchart.com/stocks/quotes/AAIN/fi...,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,AAQC,Accelerate Acquisition Corp - Class A,NYSE,2021-05-10,2022-12-15,Delisted,https://www.barchart.com/stocks/quotes/AAQC/fi...,0,0,0,...,-850,-560,-290,"-1,500","-1,260",-360,-280,-290,940,"1,170"
3,ABGI,ABG Acquisition Corp I - Class A,NASDAQ,2021-02-17,2023-02-27,Delisted,https://www.barchart.com/stocks/quotes/ABGI/fi...,0,0,0,...,-370,-270,-190,-980,-870,-410,-310,-240,450,560
4,ACACU,PLAYSTUDIOS Inc - Units (1 Ord Share Class A &...,NASDAQ,2020-10-23,2021-06-21,Delisted,https://www.barchart.com/stocks/quotes/ACACU/f...,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Parse dates & create target
Turn `ipoDate` and `delistingDate` into datetimes, and make a binary target `Delisted` (1 if status contains 'Delist', else 0).

In [2]:

def parse_date(s):
    s = str(s)
    for fmt in ["%Y-%m-%d", "%m/%d/%Y", "%Y/%m/%d", "%d-%m-%Y"]:
        try:
            return pd.to_datetime(s, format=fmt, errors="coerce")
        except Exception:
            pass
    return pd.to_datetime(s, errors="coerce")

for col in ["ipoDate", "delistingDate"]:
    if col in df.columns:
        df[col] = df[col].apply(parse_date)

# Target variable
df["Delisted"] = df.get("status", "").astype(str).str.lower().str.contains("delist").astype(int)
df["Delisted"] = df["Delisted"].fillna(0).astype(int)
df["Delisted"].value_counts(dropna=False)


Delisted
1    1617
Name: count, dtype: int64

## Clean numeric quarter fields
Convert strings like `'1,260'` to floats and coerce non-numerics to NaN.

In [3]:

import re

def to_float(x):
    if pd.isna(x): return np.nan
    s = str(x).replace(",", "").replace("$", "")
    # handle dashes
    if re.fullmatch(r"[-–—]", s):
        return np.nan
    try:
        return float(s)
    except Exception:
        return np.nan

numeric_like = [c for c in df.columns if re.search(r"(salesQ|netIncomeQ|totalAssetQ|totalLiabilitiesQ|opCashflowQ|netCashflowQ)", c, re.I)]
for c in numeric_like:
    df[c] = df[c].apply(to_float)

df[numeric_like].describe().T.head()


,count,mean,std,min,25%,50%,75%,max
salesQ5,1597.0,14813.963682,491995.058311,-9999000.0,0.0,0.0,0.0,4999740.0
salesQ4,1597.0,33264.602379,244164.262270,0.0,0.0,0.0,0.0,6565050.0
salesQ3,1597.0,32696.706324,226470.521625,0.0,0.0,0.0,0.0,4855000.0
salesQ2,1597.0,32066.161553,217931.922464,-16970.0,0.0,0.0,0.0,5523580.0
salesQ1,1597.0,31515.623043,209338.221523,-122530.0,0.0,0.0,0.0,4885680.0


## Example engineered features
- Leverage ratio: `totalLiabilitiesQ1 / totalAssetQ1`
- Profit margin proxy: `netIncomeQ1 / salesQ1`
- Cashflow ratio: `opCashflowQ1 / salesQ1`


In [4]:

eps = 1e-9
if "totalLiabilitiesQ1" in df.columns and "totalAssetQ1" in df.columns:
    df["lev_ratio_q1"] = df["totalLiabilitiesQ1"] / (df["totalAssetQ1"] + eps)

if "netIncomeQ1" in df.columns and "salesQ1" in df.columns:
    df["profit_margin_q1"] = df["netIncomeQ1"] / (df["salesQ1"] + eps)

if "opCashflowQ1" in df.columns and "salesQ1" in df.columns:
    df["cashflow_ratio_q1"] = df["opCashflowQ1"] / (df["salesQ1"] + eps)

# Keep a modeling subset (ID columns + engineered + a few raw)
keep = ["symbol", "exchange", "ipoDate", "delistingDate", "Delisted",
        "salesQ1","netIncomeQ1","totalAssetQ1","totalLiabilitiesQ1","opCashflowQ1",
        "lev_ratio_q1","profit_margin_q1","cashflow_ratio_q1"]
keep = [c for c in keep if c in df.columns]
feat_df = df[keep].copy()

feat_df.to_csv(FEAT_CSV, index=False)
print("Wrote features:", FEAT_CSV, "shape:", feat_df.shape)
feat_df.head()


Wrote features: ../data/processed/features.csv shape: (1617, 13)


,symbol,exchange,ipoDate,delistingDate,Delisted,salesQ1,netIncomeQ1,totalAssetQ1,totalLiabilitiesQ1,opCashflowQ1,lev_ratio_q1,profit_margin_q1,cashflow_ratio_q1
0,AACQU,NASDAQ,2020-07-14,2021-06-24,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AAIN,NYSE,2021-07-19,2024-01-30,1,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000e+00,0.000000e+00
2,AAQC,NYSE,2021-05-10,2022-12-15,1,0.0,5870.0,402000.0,433810.0,-1260.0,1.079129,5.870000e+12,-1.260000e+12
3,ABGI,NASDAQ,2021-02-17,2023-02-27,1,0.0,-220.0,151720.0,156170.0,-870.0,1.029330,-2.200000e+11,-8.700000e+11
4,ACACU,NASDAQ,2020-10-23,2021-06-21,1,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000e+00,0.000000e+00


✅ **Output**: `features.csv` for model training.